In [1]:
from z3 import *
from __future__ import annotations
from dataclasses import dataclass

In [2]:
import sys
sys.path.append('src')

Picking up from previously, we are going to use the definitions included with the lab starter code this time.

In [3]:
from parser import parse
from tinyscript_util import (
    stringify,
    term_enc,
    fmla_enc
)
import tinyscript as tn

Here's a quick demonstration of the parser and pretty-printer. The program on display is one of the test cases, to give an idea of what they look like.

In [4]:
with open('tests/test000.tinyscript', 'r') as f:
    prog = parse(f.read())

In [5]:
print(stringify(prog))

f := -93;
c := 7;
b := 14;
e := 59;
i := -52;
a := 22;
j := -55;
if (((-188)*(sec_4))==((134)+(g))) then
    if (((sec_1)+(120))<((149)+(-89))) then
        while (((58)*(-155))==(-238)) do
            c := sec_3
        done
    else
        if ((i)<((f)-(175))) then
            output (-248)*(200)
        else
            output b
        endif
    endif
else
    output ((-194)*(f))-((e)-(c));
    a := ((-232)*(-28))-((d)*(a));
    if ((sec_2)<(i)) then
        g := (g)*(b)
    else
        output ((127)-(-66))+((-92)*(c))
    endif
endif


Let's use a more familiar example program to demonstrate today's material. Below is an imperative fibonacci calculator.

In [6]:
fib = parse((
"""
x := 0;
y := 1;
z := 1;
i := 0;
while ((i)<(n)) do
    x := y;
    y := z;
    z := (x)+(y);
    i := (i)+(1)
done
"""))

## Implementing the box axioms

Below is our implementation of the box modality. This is refined a bit from what we did live at the end of lecture.
* Formulas are simplified aggressively at each step
* We use a slightly less complicated version of the axiom for loops

In [7]:
# Apply axioms of dynamic logic for [alpha] P
def box(alpha: tn.Prog, P: BoolRef, max_depth: int=10, depth_exceed_strict=True) -> BoolRef:
    
    if max_depth < 1:
        return z3.BoolVal(False) if depth_exceed_strict else z3.BoolVal(True)
    
    match alpha:
        
        case tn.Skip():
            return P
        
        # [x := e] P(x) <--> P(e)
        case tn.Asgn(left, right):
            return simplify(substitute(P, [(term_enc(tn.Var(left)), term_enc(right))]))
            
        # [alpha; beta] P <--> [alpha]([beta] P)
        case tn.Seq(alpha_p, beta_p):
            return simplify(box(alpha_p, 
                                box(beta_p, P, max_depth), 
                                max_depth))
            
        # [If(Q) alpha else beta] P <--> (Q -> [alpha] P) ^ (~Q -> [beta] P)
        case tn.If(Q, alpha_p, beta_p):
            return simplify(And(Implies(fmla_enc(Q), box(alpha_p, P, max_depth)),
                                Implies(fmla_enc(tn.NotF(Q)), box(beta_p, P, max_depth))))
            
        # [while(Q) alpha] P <--> (Q -> [alpha; while(Q) alpha] P) ^ (~Q -> P)
        # Note that this is equivalent to:
        # [while(Q) alpha] P <--> [if(Q) { alpha; while(Q) alpha } else { assert(True) }] P
        case tn.While(Q, alpha_p):
            return simplify(box(tn.If(Q, 
                                   tn.Seq(alpha_p, alpha),
                                   tn.Asgn('x', tn.Var('x'))), 
                                P, max_depth-1))

A basic sanity check: `[x := 1](x < 0)`

We should get `1 < 0`, which simplifies to `False`

In [8]:
alpha = tn.Asgn('x', tn.Const(1))
post = tn.LtF(tn.Var('x'), tn.Const(0))
pre = box(alpha, fmla_enc(post))
print('Program:', stringify(alpha))
print('Verification condition:', pre)

Program: x := 1
Verification condition: False


A slightly more interesting example: is 88 a Fibonacci number? If so, then the formula generated by `box` for the postcondition `x = 88` should be satisfiable.

We see that it is not, because the formula is unsatisfiable, which means that there are no inputs (i.e., assignments to `n`) that result in `x = 88` in the final state.

In [9]:
P = fmla_enc(tn.EqF(tn.Var('x'), tn.Const(88)))
vc = box(fib, P, 100)
s = Solver()
s.add(vc)
s.check()

unsat

By the same token, we can test valid Fibonacci numbers for their index: `box` essentially runs the Fibonacci program in reverse to find the initial states that yield the final states described in the postcondition.

In [10]:
P = fmla_enc(tn.EqF(tn.Var('x'), tn.Const(89)))
vc = box(fib, P, 100)
s = Solver()
s.add(vc)
s.check()

sat

We can see what satisfying assignment the solver found.

In [11]:
s.model()

[n = 11]

## A simple contract checker

Using `box`, we can also implement a (very) simple contract verifier.
The function below returns `True` if the program satisfies its contract, and if it doesn't, then it returns a counterexample input that will violate the contract.

Note the way that it works: `z3` is a *satisfiability* solver, not a validity checker. We've already seen that these semantic notions are duals of eachother, so to check for the validity of `P -> [alpha] Q` using `z3`, we negate and check for satisfiability. This negated formula is called a *verification condition*. It is unsatisfiable iff the program follows its contract, and otherwise any satisfying assignments correspond to an initial state (i.e., set of inputs) that will lead the program to violate its contract.

This type of verification is called *bounded model checking*.

In [12]:
from symbolic import Result
from tinyscript_util import (
    check_sat,
    state_from_z3_model
)

# verify contracts of the form P --> [alpha] Q
def verify_contract(
    alpha: tn.Program, 
    P: tn.Formula, Q: tn.Formula, 
    max_depth: int=10
) -> tuple[Result, tn.State]:
    weakest_pre = box(alpha, fmla_enc(Q), max_depth=max_depth)
    res, model = check_sat([Not(Implies(fmla_enc(P), weakest_pre))])
    if res == unsat:
        return (Result.Satisfies, None)
    elif res == sat:
        state = state_from_z3_model(alpha, model)
        return (Result.Violates, state)
    else:
        return (Result.Unknown, None)

The precondition below encodes that we should find the fourth Fibonacci number (0-based), and the postcondition asserts that the result should be 1. This is incorrect, as the result should be 2. The verifier should give us a counterexample.

In [13]:
pre = tn.EqF(tn.Var('n'), tn.Const(3))
post = tn.EqF(tn.Var('x'), tn.Const(1))
verify_contract(fib, pre, post)

(<Result.Violates: 2>,
 State(variables={'x': 0, 'y': 0, 'z': 0, 'i': 0, 'n': 3}))

Correcting the mistake, we see that the verifier behaves as expected.

In [14]:
pre = tn.EqF(tn.Var('n'), tn.Const(2))
post = tn.EqF(tn.Var('x'), tn.Const(1))
verify_contract(fib, pre, post)

(<Result.Satisfies: 1>, None)

## Checking Invariant Properties

Below is a sample checker that follows the analysis workflow outlined in the lab handout. It targets invariant properties, which are characterized by a formula $P$ that must remain true at all times throughout a program's execution.

$$
\Phi_P = \{\sigma : \forall i. 0\le i < |\sigma| \rightarrow \sigma_i \vDash P\}
$$

In practice, the above definition is too strict, because programs have no control over their initial state. For example, consider the invariant property which says that `x`, `y`, and `z` should all remain non-negative. The following program begins by attempting to establish the invariant.
```
x := 0;
y := 0;
z := 0;
...
```
However, it does not satisfy the invariant property, because it may begin in an initial state that maps `x`, `y`, and `z` to negative numbers, and isn't able to correct it until entering the fourth state of a trace. To address this, we will give the program a grace period in which it must establish the invariant. After it has done so, then the invariant must remain true for the remainder of the trace.

$$
\Phi_P = \{\sigma : \exists i. \sigma_i \vDash P \land \forall j. i \le j < |\sigma| \rightarrow \sigma_i \vDash P\}
$$

To enforce this property, we will add instrumentation to each instruction that could potentially either establish or violate the invariant. For the live coding exercise, we only consider assignment, composition, conditional, and while statements in our program. Among these, the only type of instruction that can affect the invariant is assignment. However, if we were considering the full tinyscript language, then we would need to think about whether `output` commands could be relevant, and how to deal with the effects of `abort`.

The instrumentation that we add will track two policy variables, `#inv_established` and `#inv_true`. `#inv_established` is initialized by a conditional statement which checks whether the initial state satisfies `P`. After this initialization, none of the instrumentation will ever set `#inv_established` to 0, reflecting that having established the invariant cannot be undone.

For each assignment instruction $\alpha$, we will determine whether $[\alpha]\,P \leftrightarrow P$. If so, then there is no need to add instrumentation. If not, then we must set the policy variables accordingly. The instrumentation constructs a box-free equivalent of $[\alpha]\,P$, and add a conditional statement that branches on this formula: the "then" branch handles the case where the invariant will be true after the instrumented instruction, and the "else" branch where it will be false.
* In the "then" branch, if `#inv_established` is currently 0, then we want to set both policy variables to 1. Add the corresponding conditional statement to construct the body of this branch.
* In the "else" branch, the instrumentation sets `#inv_true` to 0.

As we will see below, there are a few optimizations for this approach that are easy to implement, but this instrumentation is sufficient to establish the desired correspondence. If $\alpha'$ is the instrumented version of $\alpha$, then:

$$
\alpha~\text{satisfies the invariant policy for formula}~P
\quad\Longleftrightarrow\quad
\vDash [\alpha'](\mathtt{\#inv\_established} = 1 \land \mathtt{\#inv\_true} = 1)
$$

Having this, `symbolic_check` is straightforward to implement

### Implementation

We will start by writing some utility functions. When we compute the box-free equivalent of $[\alpha] P$ using `box`, the result will be a Z3 `BoolRef`. But to construct instrumentation, we need the corresponding formula as a `tn.Formula`. The utility `z3_to_fmla` below accomplishes this.

In [15]:
def z3_to_fmla(P: BoolRef):
    """
    Convert a Z3 BoolRef to a tinyscript formula
    """
    if is_int_value(P):
        return tn.Const(P.as_long())
    elif is_const(P) and not is_true(P) and not is_false(P):
        return tn.Var(str(P))
    elif is_add(P):
        return tn.Sum(z3_to_fmla(P.children()[0]),
                      z3_to_fmla(P.children()[1]))
    elif is_sub(P):
        return tn.Difference(z3_to_fmla(P.children()[0]),
                             z3_to_fmla(P.children()[1]))
    elif is_true(P):
        return tn.TrueC()
    elif is_false(P):
        return tn.FalseC()
    elif is_lt(P):
        return tn.LtF(z3_to_fmla(P.children()[0]), 
                      z3_to_fmla(P.children()[1]))
    elif is_le(P):
        return tn.OrF(tn.LtF(z3_to_fmla(P.children()[0]),
                             z3_to_fmla(P.children()[1])),
                      tn.EqF(z3_to_fmla(P.children()[0]),
                             z3_to_fmla(P.children()[1])))
    elif is_gt(P):
        return tn.NotF(tn.LtF(z3_to_fmla(P.children()[0]), 
                              z3_to_fmla(P.children()[1])))
    elif is_eq(P):
        return tn.EqF(z3_to_fmla(P.children()[0]), 
                      z3_to_fmla(P.children()[1]))
    elif is_not(P):
        return tn.NotF(z3_to_fmla(P.children()[0]))
    elif is_and(P):
        return tn.AndF(z3_to_fmla(P.children()[0]),
                       z3_to_fmla(P.children()[1]))
    elif is_or(P):
        return tn.OrF(z3_to_fmla(P.children()[0]),
                       z3_to_fmla(P.children()[1]))
    elif is_implies(P):
        return tn.ImpliesF(z3_to_fmla(P.children()[0]),
                           z3_to_fmla(P.children()[1]))
    else: 
        raise TypeError(
            f"Expected BoolRef, got {P}"
        )

We will also be checking formulas for equivalence when adding instrumentation. The utility `fmlas_equiv` does so.

In [16]:
def fmlas_equiv(P: BoolRef, Q: BoolRef) -> bool:
    """
    Test whether P and Q are equivalent, i.e., whether |= P <--> Q
    """
    res, _ = check_sat([Not(P == Q)])
    if res == sat or res == unknown:
        # If unknown, we conservatively assume that the
        # formulas are not equivalent. This will not break
        # anything, but merely result in a potentially
        # unnecessary instrumentation.
        return False
    else:
        return True

We'll define globals for the policy variables, rather than hard-coding them throughout our implementation.

In [17]:
SETUP_VAR = '#inv_established'
INV_VAR = '#inv_true'

Now for the actual instrumentation. The function `invariant_instrument` constructs the instrumentation statements to place before each assignment. It takes a single `BoolRef` argument, which should be the box-free equivalent of $[\alpha]\,P$, where $\alpha$ is the assignment being instrumented. It implements the approach described earlier, except with two optimizations.
* If the argument `Q` is the constant `False`, then it means that the invariant will certainly be violated when the assignment is executed. In this case, the instrumentation just sets `#inv_true` to 0.
* Similarly, if the argument `Q` is the constant `True`, then the invariant formula `P` will certainly be true after then assignment is executed. In this case, the instrumentation just sets the policy variables to 1 (if appropriate), and does not contain a conditional to test if `Q` is true.

In [18]:
def invariant_instrument(Q: BoolRef) -> tn.Prog:
    """
    Construct instrumentation to enforce an invariant P,
    to be placed immediately before an assignment alpha.
    
    Args:
        Q (z3.BoolRef): A box-free formula that is equivalent
            to [alpha] P.
    
    Returns:
        tn.Prog: A tinyscript program that will set the
            policy variables appropriately to enforce the
            invariant P.
    """
    true_ins = tn.If(tn.EqF(tn.Var(SETUP_VAR), tn.Const(0)),
                     tn.Seq(tn.Asgn(INV_VAR, tn.Const(1)),
                            tn.Asgn(SETUP_VAR, tn.Const(1))),
                     tn.Skip())
    false_ins = tn.Asgn(INV_VAR, tn.Const(0))
    if is_true(Q):
        return true_ins
    elif is_false(Q):
        return false_ins
    else:
        return tn.If(z3_to_fmla(Q),
                     true_ins,
                     false_ins)

Putting this all together, we can write `add_instrumentation`, which recurses on the structure of a program to add policy-state maintenance instrumentation before each assignment.

In [19]:
def add_instrumentation(alpha: tn.Prog, inv: tn.Formula) -> tn.Prog:
    """
    Construct instrumentation to enforce an invariant P,
    to be placed immediately before an assignment alpha.
    
    Args:
        alpha (tn.Prog): The program to instrument
        inv (tn.Formula): The invariant formula to enforce
    
    Returns:
        tn.Prog: A tinyscript program with instrumentation before each
            assignment.
            
    Raises:
        TypeError: The provided alpha is not a valid tinyscript program.
    """
    match alpha:
        # assignments can violate the invariant, so instrument them directly
        case tn.Asgn(name, aexp):
            pre = box(alpha, fmla_enc(inv))
            # pre will be equivalent to inv if and only if the assignment
            # has no effect on whether the invariant will be violated or
            # established, so we don't add instrumentation if this is
            # the case.
            if not fmlas_equiv(fmla_enc(inv), pre):
                ins = invariant_instrument(pre)
                if ins != tn.Skip():
                    return tn.Seq(ins, alpha)
            return alpha
        # composition cannot violate the invariant unless through either
        # of its constituents, so recurse and do not add instrumentation directly
        case tn.Seq(alpha_p, beta_p):
            ins_alpha = add_instrumentation(alpha_p, inv)
            ins_beta = add_instrumentation(beta_p, inv)
            return tn.Seq(ins_alpha, ins_beta)
        # same with conditionals
        case tn.If(p, alpha_p, beta_p):
            ins_alpha = add_instrumentation(alpha_p, inv)
            ins_beta = add_instrumentation(beta_p, inv)
            return tn.If(p, ins_alpha, ins_beta)
        # same with while loops
        case tn.While(q, alpha_p):
            ins_alpha = add_instrumentation(alpha_p, inv)
            return tn.While(q, ins_alpha)
        # skips do nothing for invariants, so no instrumentation
        case tn.Skip():
            return alpha
        case _:
            raise TypeError(
                f"instrument got {type(alpha)} ({alpha}), not Prog"
            )

def instrument(alpha: tn.Prog, invariant: tn.Formula) -> tn.Prog:
    instr = add_instrumentation(alpha, invariant)
    initialize = tn.If(invariant,
                       tn.Seq(tn.Asgn(SETUP_VAR, tn.Const(1)),
                              tn.Asgn(INV_VAR, tn.Const(1))),
                       tn.Seq(tn.Asgn(SETUP_VAR, tn.Const(0)),
                              tn.Asgn(INV_VAR, tn.Const(0))))
    return tn.Seq(initialize, instr)

Let's test it out

In [20]:
inv = tn.LtF(tn.Var('x'), tn.Sum(tn.Var('z'), tn.Const(1)))
instrumented = instrument(fib, inv)
print(stringify(instrumented))

if ((x)<((z)+(1))) then
    #inv_established := 1;
    #inv_true := 1
else
    #inv_established := 0;
    #inv_true := 0
endif;
if (!(((z)<(-1))||((z)==(-1)))) then
    if ((#inv_established)==(0)) then
        #inv_true := 1;
        #inv_established := 1
    else
        skip
    endif
else
    #inv_true := 0
endif;
x := 0;
y := 1;
if (!(((2)<(x))||((2)==(x)))) then
    if ((#inv_established)==(0)) then
        #inv_true := 1;
        #inv_established := 1
    else
        skip
    endif
else
    #inv_true := 0
endif;
z := 1;
i := 0;
while ((i)<(n)) do
    if (!(((z)<((-1)+(y)))||((z)==((-1)+(y))))) then
        if ((#inv_established)==(0)) then
            #inv_true := 1;
            #inv_established := 1
        else
            skip
        endif
    else
        #inv_true := 0
    endif;
    x := y;
    y := z;
    if (!(((y)<(-1))||((y)==(-1)))) then
        if ((#inv_established)==(0)) then
            #inv_true := 1;
            #inv_established := 1
        else
            s

Now we implement `symbolic_check`. Note that we pass `depth_exceed_strict=True` to `box`. This means that if the program could potentially run for longer than reflected after `max_depth` times unrolling the loop, then the checker will assume that the invariant could be violated on such a long-running trace.

However, the checker will not necessarily return `Result.Violates` if this happens. Rather, if the solver returns `sat`, then `symbolic_check` will attempt to generate an initial state that will lead the program to violate the invariant. If it sees that the invariant is violated in the final state (i.e., `#inv_true` is 0), then it returns `Result.Violates`. Otherwise it returns `Result.Unknown`.

In [21]:
from interpreter import exc
from symbolic import Result

def symbolic_check(alpha: tn.Prog, P: tn.Formula, max_depth=10, timeout=10) -> Result:
    alpha_p = instrument(alpha, P)
    post = tn.AndF(tn.EqF(tn.Var(SETUP_VAR), tn.Const(1)),
                   tn.EqF(tn.Var(INV_VAR), tn.Const(1)))
    weakest_pre = box(alpha, fmla_enc(post), max_depth=max_depth, depth_exceed_strict=True)
    
    res, model = check_sat([Not(weakest_pre)], timeout=timeout)
    
    if res == unsat:
        return Result.Satisfies
    elif res == sat:
        state = state_from_z3_model(alpha, model, complete=True)
        final_state = exc(state, alpha_p, max_steps=1.e6, quiet=False)
        if final_state[0].variables[INV_VAR] == 0:
            return Result.Violates
    return Result.Unknown

On our example, we see that the checker isn't able to determine that the invariant will always hold, but it is unable to find a counterexample that concretely demonstrates a violation. It returns unknown.

In [22]:
symbolic_check(fib, inv)

<Result.Unknown: 3>

If we change the invariant to `x <= n`, then we should get a violation.

In [23]:
symbolic_check(fib, tn.LtF(tn.Var('x'), tn.Sum(tn.Var('n'), tn.Const(1))), max_depth=15)

<Result.Violates: 2>